In [1]:
import numpy as np
import cv2
import pytesseract
#from matplotlib import pyplot as plt

In [2]:
def image_rotate(img_path):
    ele_size=(8,2)
    image = cv2.imread(img_path)
    img = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    img_sobel = cv2.Sobel(img,cv2.CV_8U,1,0)#same as default,None,3,1,0,cv2.BORDER_DEFAULT)
    img_threshold = cv2.threshold(img_sobel,0,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY)
    element = cv2.getStructuringElement(cv2.MORPH_RECT,ele_size)
    thresh = cv2.morphologyEx(img_threshold[1],cv2.MORPH_CLOSE,element)
    coords = np.column_stack(np.where(thresh > 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h),flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

In [20]:
def resize_image(img,dx,dy):
    # scale_percent:percent of original size
#     width = int(img.shape[1] * scale_percent / 100)
#     height = int(img.shape[0] * scale_percent / 100)
#     dim = (width, height)
    # resize image
    resized = cv2.resize(img, None,fx=dx, fy=dy , interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(resized, './resized.jpg')
    return resized

In [4]:
def text_detect(img,ele_size=(8,2)): #
    if len(img.shape)==3:
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_sobel = cv2.Sobel(img,cv2.CV_8U,1,0)#same as default,None,3,1,0,cv2.BORDER_DEFAULT)
    img_threshold = cv2.threshold(img_sobel,0,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY)
    element = cv2.getStructuringElement(cv2.MORPH_RECT,ele_size)
    img_threshold = cv2.morphologyEx(img_threshold[1],cv2.MORPH_CLOSE,element)
    res = cv2.findContours(img_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if cv2.__version__.split(".")[0] == '3':
        _, contours, hierarchy = res
    else:
        contours, hierarchy = res
    Rect = [cv2.boundingRect(i) for i in contours if i.shape[0]>100]
    RectP = [(int(i[0]-i[2]*0.08),int(i[1]-i[3]*0.1),int(i[0]+i[2]*1.1),int(i[1]+i[3]*1.2)) for i in Rect]
    return RectP

In [5]:
def OCR_image(img):
    return pytesseract.image_to_string(img, lang='kor+eng')

In [6]:
def OCR_detected_image(img, RectP):
    for j in RectP:
        roi = img[j[1]:j[3],:]
        try:
            print(pytesseract.image_to_string(roi, lang='kor'))
        except:
            pass
    